## Embedding Word into Vectors
- Article that generates the idea: [Neural Word Embedding as Implicit Matrix Factorization](http://u.cs.biu.ac.il/~nlp/wp-content/uploads/Neural-Word-Embeddings-as-Implicit-Matrix-Factorization-NIPS-2014.pdf)
- Text8 used by google's word2vec [text8 data](http://mattmahoney.net/dc/text8.zip)
- [wordsim353](http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/)
- [MEN Test collection](http://clic.cimec.unitn.it/~elia.bruni/MEN)

## Algorithm Design Considerations
- we extend the original algorithm from allowing a fixed length window to a set of different length windows - not sure whether it will improve the result
- the algorithm filter out low freq words and contexts. Current impl may result in all-zero rows in the matrix when the row is a frequent word that is linked to a lot of infrequent contexts - this is very unlikely though.
- we count the word frequency once for different windows, it will only affect the frequency filter for word

## Experimental Observations

In [3]:
## load text data
import re
corpus = open("data/text8").read()
#%time corpus = corpus.replace(".", " DOT DOT")
%time corpus = re.findall(r"\w+", corpus)
print len(corpus)

CPU times: user 2.65 s, sys: 253 ms, total: 2.9 s
Wall time: 2.9 s
17005207


In [4]:
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import check_array, as_float_array, check_random_state
from sklearn.utils.extmath import randomized_svd, safe_sparse_dot, svd_flip
from sklearn.utils.sparsefuncs import mean_variance_axis

import scipy.sparse as sp
class SymmetricSVD(TruncatedSVD):
    def __init__(self, n_components=2, algorithm="randomized", n_iter=5,
                    random_state=None, tol=0.):
        super(SymmetricSVD, self).__init__(n_components, algorithm, 
                                          n_iter, random_state, tol)
    def fit_transform(self, X, y=None):
        """
        After svd, we have M = U * Sigma * VT, traditional SVD
        return W = U * Sigma as the transformed vectors; here 
        in SymmetricSVD version, it is W = U * sqrt(Sigma) that is returned
        as the transformed vectors.
        In the paper [Neural Word Embedding as Implicit Matrix Factorization], the 
        authors claim that it is not clear why this works better, but it does work
        better than the traditional approach.
        
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Training data.
        Returns
        -------
        X_new : array, shape (n_samples, n_components)
            Reduced version of X. This will always be a dense array.
        """
        X = as_float_array(X, copy=False)
        random_state = check_random_state(self.random_state)

        # If sparse and not csr or csc, convert to csr
        if sp.issparse(X) and X.getformat() not in ["csr", "csc"]:
            X = X.tocsr()

        if self.algorithm == "arpack":
            U, Sigma, VT = svds(X, k=self.n_components, tol=self.tol)
            # svds doesn't abide by scipy.linalg.svd/randomized_svd
            # conventions, so reverse its outputs.
            Sigma = Sigma[::-1]
            U, VT = svd_flip(U[:, ::-1], VT[::-1])

        elif self.algorithm == "randomized":
            k = self.n_components
            n_features = X.shape[1]
            if k >= n_features:
                raise ValueError("n_components must be < n_features;"
                                 " got %d >= %d" % (k, n_features))
            U, Sigma, VT = randomized_svd(X, self.n_components,
                                          n_iter=self.n_iter,
                                          random_state=random_state)
        else:
            raise ValueError("unknown algorithm %r" % self.algorithm)

        self.components_ = VT

        # Calculate explained variance & explained variance ratio
        ## USE SQRT OF SIGMA INSTEAD OF SIGMA ITSELF
        X_transformed = np.dot(U, np.sqrt(np.diag(Sigma)))
        self.explained_variance_ = exp_var = np.var(X_transformed, axis=0)
        if sp.issparse(X):
            _, full_var = mean_variance_axis(X, axis=0)
            full_var = full_var.sum()
        else:
            full_var = np.var(X, axis=0).sum()
        self.explained_variance_ratio_ = exp_var / full_var
        return X_transformed

In [5]:
from collections import Counter
from scipy import sparse
import numpy as np
import pandas as pd

class MFWordEmbedder(object):
    """Matrix Factorization Word Embedder
    """
    def __init__(self, min_wf = 120, min_cf = 6, window = 2, nneg = 1., 
                 svdtype="truncated", vec_dim = 100):
        """
        Parameters: 
        min_wf - minimum number of frequence for a word to be modelled
        min_cf - minimum number of frequence for a context (word window) to be modelled
        window - int or list, length(s) for word seq for context
        nneg - float number of negative sample, plays as the offset in the matrix factorization here
        svdtype - string {"symmetric", "truncated"}
        vec_dim - int: dimension of learned word vectors
        """
        self.min_wf = min_wf
        self.min_cf = min_cf
        self.window = window
        self.nneg = nneg
        self.svdtype = svdtype
        self.vec_dim = vec_dim
    
    def build_hash(self, words):
        windows = [self.window] if isinstance(self.window, int) else sorted(self.window)
        word_hash, nwords = {}, 0
        context_hash, ncontexts = {}, 0
        # row/col for matrix - only words >= min_wf and contexts >= min_cf will get in
        wordhash2row, nrows = {}, 0 
        contexthash2col, ncols = {}, 0
        ## (wordhash, contexthash) pairs
        wh_ch_pairs = []
        
        for iw, window in enumerate(windows):
            for i in xrange(window, len(words)-window):
                word, context = words[i], tuple(words[i-window:i]+words[i+1:i+window+1])
                ## only update word hash for the first window scanning
                if (iw == 0): 
                    if word in word_hash:
                        h, n = word_hash[word]
                        word_hash[word] = (h, n+1)
                    else:
                        word_hash[word] = (nwords, 1)
                        nwords += 1
                    ## update rows if word occure frequent enought
                    if word_hash[word][1] == self.min_wf: 
                        wordhash2row[word_hash[word][0]] = nrows
                        nrows += 1
                ## update context hash
                if context in context_hash:
                    h, n = context_hash[context]
                    context_hash[context] = (h, n+1)
                else:
                    context_hash[context] = (ncontexts, 1)
                    ncontexts += 1
                ## update cols if context occure frequently enough
                if context_hash[context][1] == self.min_cf:
                    contexthash2col[context_hash[context][0]] = ncols
                    ncols += 1
                ## update wordhash, contexthash pairs
                wh_ch_pairs.append( (word_hash[word][0], context_hash[context][0]) )
        
        wh_ch_pairs = np.array([(wid, cid) for wid, cid in wh_ch_pairs 
                                           if wid in wordhash2row if cid in contexthash2col])
        inv_word_hash = dict([(h,w) for w,(h,n) in word_hash.items()])
        row2hash = dict([(r, h) for h, r in wordhash2row.items()])
        self.words = np.array([inv_word_hash[row2hash[i]] for i in xrange(len(row2hash))])
        #self.row2word = dict([(r, inv_word_hash[h]) for h, r in wordhash2row.items()])
        #self.word2row = dict([(v,k) for k,v in self.row2word.items()])
        return word_hash, context_hash, wordhash2row, contexthash2col, wh_ch_pairs
    
    def build_matrix(self, wordhash2row, contexthash2col, wh_ch_pairs):
        wordids, contextids = zip(*wh_ch_pairs)
        widcounter = Counter(wordids)
        cidcounter = Counter(contextids)
        npairs = len(wh_ch_pairs) * 1.
        nrows, ncols = len(wordhash2row), len(contexthash2col)
        
        ## some rows in M might be all-zero if a frequent word is linked to a lot of infrequent 
        ## contexts, even though it is expected to be rare in practice
        data = np.array([npairs / widcounter[wid] / cidcounter[cid] for wid, cid in wh_ch_pairs])
        rows = np.array([wordhash2row[wid] for wid in wordids])
        cols = np.array([contexthash2col[cid] for cid in contextids])
        
        M = sparse.coo_matrix( (data, (rows, cols)), shape = (nrows, ncols), dtype=np.float32 )
        M.data = np.log(M.data) - np.log(self.nneg)
        M = M.tocsr()
        M[M<0.0] = 0.0
        return M
    
    def svd(self, M, *args, **kwargs):
        svd = (TruncatedSVD(n_components=self.vec_dim, *args, **kwargs) 
               if self.svdtype == "truncated" 
               else SymmetricSVD(n_components=self.vec_dim, *args, **kwargs))
        word_vectors = svd.fit_transform(M)
        return word_vectors
    
    def fit(self, words):
        word_hash, context_hash, wordhash2row, contexthash2col, wh_ch_pairs = self.build_hash(words)
        M = model.build_matrix(wordhash2row, contexthash2col, wh_ch_pairs)
        self.word_vectors = self.svd(M)
        return self
    
    def get_word_vectors(self):
        return pd.DataFrame(data = self.word_vectors, index = self.words, copy=False)

In [6]:
model = MFWordEmbedder(min_wf=2, min_cf=1, window=2, vec_dim=2)
a = corpus[:10] * 2
print a
model.fit(a)

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [7]:
#model = MFWordEmbedder(svdtype="truncated", window=2, min_cf=4, vec_dim=100)
model = MFWordEmbedder(svdtype="symmetric", window=2, min_cf=3, vec_dim=100)
%time model.fit(corpus)

CPU times: user 2min 2s, sys: 23.2 s, total: 2min 25s
Wall time: 1min 28s


In [8]:
%time wvs = model.get_word_vectors()
wvs.loc[["king", "queen"]]

CPU times: user 1.09 ms, sys: 389 µs, total: 1.48 ms
Wall time: 1.42 ms


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
king,0.000052,0.000788,0.001362,0.001186,0.000378,0.002197,0.001757,0.008872,0.011870,-0.000413,...,-0.081232,-0.010924,-0.151333,0.067235,-0.172565,0.067491,0.021508,-0.054948,0.047741,0.078047
queen,0.000021,0.000160,0.000357,0.000888,0.000292,0.000574,0.000179,0.004333,0.005334,-0.000206,...,-0.018251,-0.012156,-0.058302,0.027453,-0.037100,0.023543,0.002896,-0.012540,-0.000916,0.015875


In [9]:
sum(wvs.sum(axis = 1)==0), wvs.shape[0]

(179, 10401)

In [10]:
## use flann to find knn
import pyflann as pf
from scipy import stats

class NearestNeighbor(object):
    def __init__(self, k = 5, algorithm="kdtree", distance_type="euclidean"):
        pf.set_distance_type(distance_type)
        self.flann = pf.FLANN()
        self.k = k
        self.algorithm = "autotuned"#algorithm
        self.iterations = 100
    def train(self, X):
        self.X_ = X
    def nearest(self, X):
        min_index, dists = self.flann.nn(self.X_, X, self.k, 
                                         algorithm = self.algorithm, 
                                         iterations=self.iterations)
        return min_index, dists

In [11]:
nn=NearestNeighbor()
nn.train(model.word_vectors)
%time neighbors, dists =nn.nearest(model.word_vectors)

CPU times: user 1min 13s, sys: 1.99 s, total: 1min 15s
Wall time: 1min 14s


In [12]:
for row in neighbors:
    close_words = [model.words[r] for r in row]
    if close_words[0] == "east":
        print close_words

['east', 'west', 'south', 'north', 'central']
